# Le Robo Reviews Start

In [1]:
import pandas as pd
from transformers import BertForSequenceClassification, BertTokenizer
from sklearn.cluster import KMeans
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [2]:
import pandas as pd

# Load the three CSV files into DataFrames
df1 = pd.read_csv('Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')
df2 = pd.read_csv('Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv')
df3 = pd.read_csv('1429_1.csv')

# Concatenate the DataFrames
combined_df = pd.concat([df1, df2, df3], ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('combined_amazon_reviews.csv', index=False)


C:\Users\diego\AppData\Local\Temp\ipykernel_7848\268574199.py:6: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('1429_1.csv')


In [3]:
import pandas as pd

df = pd.read_csv('combined_amazon_reviews.csv')
print(df.columns)


Index(['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand',
       'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer',
       'manufacturerNumber', 'reviews.date', 'reviews.dateSeen',
       'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs',
       'reviews.dateAdded', 'reviews.userCity', 'reviews.userProvince'],
      dtype='object')


C:\Users\diego\AppData\Local\Temp\ipykernel_7848\699921908.py:3: DtypeWarning: Columns (1,2,3,7,8,11,14,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('combined_amazon_reviews.csv')


In [4]:
# Display the first 5 rows of the combined dataset
print(combined_df.head())


                     id             dateAdded           dateUpdated  \
0  AVpgNzjwLJeJML43Kpxn  2015-10-30T08:59:32Z  2019-04-25T09:08:16Z   
1  AVpgNzjwLJeJML43Kpxn  2015-10-30T08:59:32Z  2019-04-25T09:08:16Z   
2  AVpgNzjwLJeJML43Kpxn  2015-10-30T08:59:32Z  2019-04-25T09:08:16Z   
3  AVpgNzjwLJeJML43Kpxn  2015-10-30T08:59:32Z  2019-04-25T09:08:16Z   
4  AVpgNzjwLJeJML43Kpxn  2015-10-30T08:59:32Z  2019-04-25T09:08:16Z   

                                                name                  asins  \
0  AmazonBasics AAA Performance Alkaline Batterie...  B00QWO9P0O,B00LH3DMUO   
1  AmazonBasics AAA Performance Alkaline Batterie...  B00QWO9P0O,B00LH3DMUO   
2  AmazonBasics AAA Performance Alkaline Batterie...  B00QWO9P0O,B00LH3DMUO   
3  AmazonBasics AAA Performance Alkaline Batterie...  B00QWO9P0O,B00LH3DMUO   
4  AmazonBasics AAA Performance Alkaline Batterie...  B00QWO9P0O,B00LH3DMUO   

          brand                                         categories  \
0  Amazonbasics  AA,AAA,Heal

In [5]:
# Check the number of rows and columns
print(f"Dataset shape: {combined_df.shape}")


Dataset shape: (67992, 27)


In [6]:
# Check for missing values in each column
missing_values = combined_df.isnull().sum()
print("Missing values in each column:\n", missing_values)

Missing values in each column:
 id                          0
dateAdded               34660
dateUpdated             34660
name                     6760
asins                       2
brand                       0
categories                  0
primaryCategories       34660
imageURLs               34660
keys                        0
manufacturer                0
manufacturerNumber      34660
reviews.date               39
reviews.dateSeen            0
reviews.didPurchase     67982
reviews.doRecommend     12840
reviews.id              67921
reviews.numHelpful      12746
reviews.rating             33
reviews.sourceURLs          0
reviews.text                1
reviews.title              19
reviews.username           13
sourceURLs              34660
reviews.dateAdded       42901
reviews.userCity        67992
reviews.userProvince    67992
dtype: int64


In [7]:
# Get information about the dataset, including data types
print(combined_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67992 entries, 0 to 67991
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    67992 non-null  object 
 1   dateAdded             33332 non-null  object 
 2   dateUpdated           33332 non-null  object 
 3   name                  61232 non-null  object 
 4   asins                 67990 non-null  object 
 5   brand                 67992 non-null  object 
 6   categories            67992 non-null  object 
 7   primaryCategories     33332 non-null  object 
 8   imageURLs             33332 non-null  object 
 9   keys                  67992 non-null  object 
 10  manufacturer          67992 non-null  object 
 11  manufacturerNumber    33332 non-null  object 
 12  reviews.date          67953 non-null  object 
 13  reviews.dateSeen      67992 non-null  object 
 14  reviews.didPurchase   10 non-null     object 
 15  reviews.doRecommend

In [8]:
# Check unique values in the product category (if available)
if 'product_category' in combined_df.columns:
    unique_categories = combined_df['product_category'].unique()
    print(f"Unique product categories: {len(unique_categories)}")
    print(unique_categories)

# Check unique values in the rating column (if available)
if 'star_rating' in combined_df.columns:
    rating_distribution = combined_df['star_rating'].value_counts()
    print(f"Rating distribution:\n{rating_distribution}")


In [9]:
def preprocess_data(data_file):
    df = pd.read_csv(data_file)
    # Inspect the column names and update the code accordingly
    print(df.columns)
    
    # Clean and preprocess the review text
    df['review_text'] = df['CORRECT_COLUMN_NAME'].apply(clean_text)
    
    # Split the dataset into train, validation, and test sets
    train_df, val_df, test_df = split_data(df)
    return train_df, val_df, test_df
